In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

In [2]:
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')

In [3]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + [y[1], ] +  y[2].split(',')[2:]

In [4]:
def mapreduce_join_util(x):
#     print x[0], x[1]
    first_d = []
    second_d = []
    for u, v in x[1]:
        if u == 1:
            first_d.append(v)
        else:
            second_d.append(v)
    return [(u, v) for u in first_d for v in second_d]

In [5]:
def mapreduce_join(rdd1, rdd2):
    rdd1_with_record_type = rdd1.map(lambda x: (x[0], [(1, x[1])]))
    rdd2_with_record_type = rdd2.map(lambda x: (x[0],[(2, x[1])]))
    combined_rdd = rdd1_with_record_type.union(rdd2_with_record_type).reduceByKey(lambda x,y : x+y)
    return combined_rdd.flatMap(mapreduce_join_util)

In [6]:
#Get RDD of all movies
movies_cleaned = df_movies.map(clean_df_movies)
all_movieids = movies_cleaned.map(lambda x: (x[0], 0))

In [7]:
#Get RDD of rated movies
movieid_rating = df_rating.map(lambda x: x.split(",")).map(lambda x:(x[1], 1)).reduceByKey(lambda x,y: 1)

In [8]:
allmovies_ratedmovies = all_movieids.union(movieid_rating)

In [9]:
unrated_movies = allmovies_ratedmovies.reduceByKey(lambda x, y: x+y).flatMap(lambda x : [x] if x[1]==0 else [])

In [10]:
unrated_movies = unrated_movies.map(lambda x: (x[0], ''))

In [11]:
movies_names = movies_cleaned.map(lambda x: (x[0],  str(x[1])))

In [12]:
combined_rdd = mapreduce_join(unrated_movies, movies_names)

In [13]:
unrated_movies_names = combined_rdd.map(lambda x: x[1])

In [14]:
unrated_movies_names.collect()

['MURDER and murder (1996)',
 'Nothing Personal (1995)',
 'Very Natural Thing, A (1974)',
 'Wedding Bell Blues (1996)',
 'Scream of Stone (Schrei aus Stein) (1991)',
 'Next Step, The (1995)',
 'Tainted (1998)',
 'Ripe (1996)',
 'Walk in the Sun, A (1945)']

In [15]:
#count of unrated movies, just for verification
all_movieids.count() - movieid_rating.count()

9